<a href="https://colab.research.google.com/github/noanocode/kotoba-whisper-v1.1/blob/main/kotoba_whisper_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ディレクトリ構造**
以下のディレクトリ構造でGoogleDrive上にフォルダを作成したのち、コードを実行していく
* **MyDrive**
    * **whispertest** : 本プロジェクトフォルダ
        * **audiofile** : 音声ファイルを格納するフォルダ
            * audio.m4a : アップロードするファイル、スマートフォンで録音したファイル形式のデフォルト拡張子
            * audio.mp3 : コード内で生成されるファイル、whisperモデルの入力拡張子に合わせてmp3を採用
        * **kotoba-whisper-v1.1.ipynb** : 作業用notebookファイル

In [ ]:
#@title ##パッケージインストール
#https://huggingface.co/kotoba-tech/kotoba-whisper-v1.1

# 必要なライブラリをインストール
#!pip install --upgrade pip
!pip install --upgrade transformers accelerate torchaudio
!pip install stable-ts==2.16.0
!pip install punctuators==0.0.5
!pip install ffmpeg
!pip install pykakasi
!pip install requests

# Google Driveにアクセスしてファイルを取得
import os
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylin

In [ ]:
#@title ##Definisionの定義
import subprocess
import torch
from transformers import pipeline
from pykakasi import kakasi
from difflib import SequenceMatcher
import re
from IPython.display import display, HTML
from google.colab import files
import requests



# カレントディレクトリを以下のパスに変更
os.chdir("/content/drive/MyDrive/whispertest")

# 入力ファイルと出力ファイルのパス設定
input_file = "/content/drive/MyDrive/whispertest/audiofile/audio.m4a"
output_file = "/content/drive/MyDrive/whispertest/audiofile/audio.mp3"

# アップロードされたファイルを指定のリンクにアップロードする関数
def upload_file_to_url(filename, url):
    with open(filename, 'rb') as f:
        files = {'file': (filename, f)}
        response = requests.post(url, files=files)
        return response

# ffmpegを使用してm4aをmp3に変換
def convert_m4a_to_mp3(input_file, output_file):
    try:
        subprocess.run(["ffmpeg", "-i", input_file, "-vn", output_file], check=True)
        print("Conversion to mp3 completed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred during conversion: {e}")

# Kotoba-Whisperを使用して音声ファイルを文字起こし
def transcribe_audio(file_path):
    model_id = "kotoba-tech/kotoba-whisper-v1.1"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model_kwargs = {"attn_implementation": "sdpa"} if torch.cuda.is_available() else {}
    generate_kwargs = {"language": "japanese", "task": "transcribe"}

    # パイプラインのセットアップ
    pipe = pipeline(
        model=model_id,
        torch_dtype=torch_dtype,
        device=device,
        model_kwargs=model_kwargs,
        chunk_length_s=15,
        batch_size=16,
        trust_remote_code=True,
        stable_ts=True,
        punctuator=True
    )

    # 音声ファイルの読み込みと推論実行
    result = pipe(file_path, return_timestamps=True, generate_kwargs=generate_kwargs)
    return result['text']

#kotoba-whisperは句読点が入ってしまうため、除去する
def remove_punctuation(text):
    # 句読点を除去するための正規表現パターン
    pattern = r'[。、．，]'
    # 句読点を空文字に置換して除去
    return re.sub(pattern, '', text)

# 特定の単語が文字起こし結果に含まれているかをチェック
def check_word_in_transcription(transcription, word):
    return word == transcription

# 日本語をローマ字に変換する関数
def japanese_to_romaji(text):
    # kakasiオブジェクトを作成
    kakasi_obj = kakasi()
    # ローマ字モードに設定
    kakasi_obj.setMode('H', 'a')  # Hiragana to romaji
    kakasi_obj.setMode('K', 'a')  # Katakana to romaji
    kakasi_obj.setMode('J', 'a')  # Japanese (Kanji) to romaji
    # 設定を反映
    kakasi_obj.setMode('r', 'Hepburn')  # ローマ字の変換方法をHepburn式に設定
    # kakasiオブジェクトを使ってテキストをローマ字に変換
    conv = kakasi_obj.getConverter()
    result = conv.do(text)
    return result

# 文字の一致率を決める関数
def is_similar_enough(string1, string2):
    # SequenceMatcherオブジェクトを作成して類似度を計算
    similarity = SequenceMatcher(None, string1, string2).ratio()
    # 類似度が閾値以上であればTrueを返す
    return similarity



In [ ]:
#@title ##Voice prediction

#音声の正解文章
record_content = "今日のご飯はハンバーグ" #@param {type:"string"}

# カレントディレクトリを以下のパスに変更
os.chdir("/content/drive/MyDrive/whispertest/audiofile")
# 指定のリンク（アップロード先のURL）を設定
upload_url = '/content/drive/MyDrive/whispertest/audiofile'
# フォルダ内のすべてのファイルを削除
for filename in os.listdir(upload_url):
    file_path = os.path.join(upload_url, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)  # ファイルを削除
        elif os.path.isdir(file_path):
            os.rmdir(file_path)  # ディレクトリを削除（空の場合）
    except Exception as e:
        print(f'削除中にエラーが発生しました: {e}')
#ファイルのアップロードボタン（ファイル名はaudio.m4aとする）
uploaded = files.upload()
# カレントディレクトリを以下のパスに変更
os.chdir("/content/drive/MyDrive/whispertest")



#関数の実行
wordpredict = japanese_to_romaji(record_content)
convert_m4a_to_mp3(input_file, output_file)
transcription = transcribe_audio(output_file)
transcription_en = japanese_to_romaji(transcription)
transcription_en = remove_punctuation(transcription_en)
is_word_present = check_word_in_transcription(transcription_en, wordpredict)
word_similarity = is_similar_enough(transcription_en, wordpredict)

#@title 結果の表示
html_content = f"""
<h2>結果の表示</h2>
<ul>
    <li><b>正解文章：</b>{wordpredict}</li>
    <li><b>文字おこし文章：</b>{transcription_en}</li>
    <li><b>結果：</b>{is_word_present}</li>
    <li><b>類似度：</b>{word_similarity}</li>
</ul>
"""
display(HTML(html_content))

##参考

In [ ]:
#「このファイルが単独で実行された場合((cmd)python xxx.py)にのみ、以降の処理を実行する」という条件分岐
if __name__ == "__main__":
    convert_m4a_to_mp3(input_file, output_file)
    transcription = transcribe_audio(output_file)
    transcription_en = japanese_to_romaji(transcription)
    transcription_en = remove_punctuation(transcription_en)
    wordpredict = "sawadasa"
    is_word_present = check_word_in_transcription(transcription_en, wordpredict)
    word_similarity = is_similar_enough(transcription_en, wordpredict)
    print(f"正解文章：{wordpredict}")
    print(f"文字おこし文章：{transcription_en}")
    print(is_word_present)
    print(f"類似度：{word_similarity}")


In [ ]:
#結果の表示方法
print(f"正解文章：{wordpredict}")
print(f"文字おこし文章：{transcription_en}")
print(f"結果：{is_word_present}")
print(f"類似度：{word_similarity}")